# Кластеризация на примере метода k средних

Ищем в данных скрытые закономерности, а точнее "ценрты скопления" данных.

1. Ищем $k$ кластеров с центрами в $\mu_i, i = 1, ..., k$.

2. Если объект $x$ ближе всего к центру $\mu_j$, относим его к кластеру $S_j$

Качество измеряем функцией $J$:

$$ J = \sum_{i = 1}^{k} \sum_{x \in S_i} \Vert x - \mu_i \Vert^2 \rightarrow \min $$

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
%matplotlib inline
from sklearn.datasets import make_blobs

X, _ = make_blobs(n_samples = 300, n_features = 2, centers = 3, cluster_std = 1.4)
plt.figure(figsize = (12, 12))
plt.axis('off')
plt.scatter(X[:, 0], X[:, 1], c = (.2, .8, .5));

Сходится долго, всякий раз в разные точки.

In [ ]:
from sklearn.cluster import KMeans

k_means = KMeans(n_clusters = 5)
k_means.fit(X);

In [ ]:
cc = k_means.cluster_centers_

plt.figure(figsize = (12, 12))
plt.axis('off')
plt.scatter(X[:, 0], X[:, 1], c = (.2, .8, .5))
plt.scatter(cc[:, 0], cc[:, 1], s = 200, marker = (5, 1), c = (.8, .2, .4));

Вопрос выбора $k$ остается открытым.

In [ ]:
from sklearn.grid_search import GridSearchCV

params = {'n_clusters' : [1, 2, 3, 4, 5]}
algorithm = GridSearchCV(KMeans(), params)
algorithm.fit(X)
algorithm.grid_scores_

Чем больше $k$, тем выше качество. Когда же остановиться?

# Сжатие изображений

В следующем примере объекты выборки - точки изображения, признаки - цвета точек.

In [ ]:
img=mpimg.imread('image.jpg')[..., 1]
plt.figure(figsize = (20, 20))
plt.axis('off')
plt.imshow(img, cmap = 'gray');

Кластеризация позволяет снизить количество цветов на изображении. Больше кластеров - меньше потеря качества, но медленнее алгоритм.

In [ ]:
from sklearn.cluster import MiniBatchKMeans
from scipy.stats import randint


X = img.reshape((-1, 1))
k_means = MiniBatchKMeans(n_clusters=3)
k_means.fit(X) 
values = k_means.cluster_centers_
labels = k_means.labels_
img_compressed = values[labels].reshape(img.shape)
plt.figure(figsize = (20, 20))
plt.axis('off')
plt.imshow(img_compressed, cmap = 'gray');

# Нахождение тем в текстах

Скрытые закономерности могут весьма содержательными.

Применим метод для кластеризации текстов. Сначала возьмем новостные данные и выбирем из них четыре конкретных категории текстов.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.cluster import KMeans
from time import time

categories = [
    'alt.atheism',
    'talk.religion.misc',
    'comp.graphics',
    'sci.space',
]

print("Loading 20 newsgroups dataset for categories:")
print(categories)

dataset = fetch_20newsgroups(subset='all', categories=categories,
                             shuffle=True, random_state=42)

print("%d documents" % len(dataset.data))
print("%d categories" % len(dataset.target_names))
print()

labels = dataset.target
true_k = np.unique(labels).shape[0]

Закодируем тексты с помощью вещественных признаков:

In [ ]:
print("Extracting features from the training dataset using a sparse vectorizer")
t0 = time()

vectorizer = TfidfVectorizer(max_df=0.5, max_features=1000,
                             min_df=2, stop_words='english',
                             use_idf=True)

X = vectorizer.fit_transform(dataset.data)

print("done in %fs" % (time() - t0))
print("n_samples: %d, n_features: %d" % X.shape)
print()

И применим к получившимся векторам метод $k$ средних.

In [ ]:
km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)

print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))
print()

print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(labels, km.labels_))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, km.labels_, sample_size=1000))

print()

order_centroids = km.cluster_centers_.argsort()[:, ::-1]

terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % (i + 1), end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()

Выше приведены слова, соответствующие самым весомым компонентам центров кластеров - самые значимые в кластере слова.

Находя их, алгоритм совершенно ничего не знал о тематике текстов. Оказались ли подобранные им темы смешанными?